In [1]:
from dotenv import load_dotenv
import os 
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import hopsworks
import re 
import modal 
#prepocessing
import requests
import pandas as pd
import json
#import pandas_market_calendars as mcal
import datetime
import numpy as np
from datetime import timedelta 


In [2]:
load_dotenv()

api_key = os.environ.get('stocks_api') # Replace this with your actual API key
ts = TimeSeries(key=api_key, output_format='pandas')

# Fetch daily adjusted stock prices; adjust the symbol as needed
data, meta_data = ts.get_daily(symbol='TSLA', outputsize='full')

print(data.head())

            1. open  2. high    3. low  4. close    5. volume
date                                                         
2024-04-30   186.98   190.95  182.8401    183.28  127031787.0
2024-04-29   188.42   198.87  184.5400    194.05  243869678.0
2024-04-26   168.85   172.12  166.3700    168.29  109815725.0
2024-04-25   158.96   170.88  158.3600    170.18  126427521.0
2024-04-24   162.84   167.97  157.5100    162.13  181178020.0


In [ ]:
def create_tsla_history():

    start_date = datetime.datetime.strptime('2015-07-16',"%Y-%m-%d")
    end_date = datetime.datetime.strptime('2023-01-05',"%Y-%m-%d")

    # Get the TSLA stock data from yfinance
    tsla = yf.Ticker("TSLA") #VEFAB.ST
    # info = tsla.info

    # get historical market data
    data = tsla.history(start=start_date, end=end_date)

    # drop some columns
    tesla_df = data.drop(columns=['Dividends','Stock Splits'])
    tesla_df.index = tesla_df.index.strftime('%Y-%m-%d')
    
    print('Number of business days included in data set: ',np.shape(tesla_df))

    # Create an array of all dates in the specified period
    all_dates = np.array([start_date + datetime.timedelta(days=i) for i in range((end_date - start_date).days)])
    all_dates = [d.strftime('%Y-%m-%d') for d in all_dates]

    # Use setdiff1d() to find the non-business days
    isBusinessDay, _ = extract_business_day(start_date='2015-07-16',end_date='2023-01-04')
    non_business_days = np.setdiff1d(all_dates, isBusinessDay)

    # Add nan-values to the non-business days
    print('Add {} non business days with NaN-values'.format(len(non_business_days)))
    for d in non_business_days:
        tesla_df.loc[d,:] = [np.nan,np.nan,np.nan,np.nan,np.nan]

    # sort index (dates)
    tesla_df = tesla_df.sort_index()
 
    # move "date"-index into its own column
    tesla_df = tesla_df.reset_index()
    
    # Rename column 'Date' to 'date'
    tesla_df = tesla_df.rename(columns={'Date': 'date'})
    print('Final size of dataframe',np.shape(tesla_df))
    
    # Write the merged dataframe to a CSV file
    start_date ='2015-07-16'
    end_date = '2023-01-05'
    save_path = "data/stock/tesla_{}-{}.csv".format(start_date,end_date)
    
    print('Save at :',save_path)
    tesla_df.to_csv(save_path, index=False)
    
    return tesla_df

In [ ]:
def extract_business_day(start_date,end_date):
    """
    Given a start_date and end_date.
    
    `Returns`:
    
    isBusinessDay: list of str (with all dates being business days)
    is_open: boolean list
        e.g is_open = [1,0,...,1] means that start_date = open, day after start_date = closed, and end_date = open
    """
    
    # Save for later
    end_date_save = end_date
    
    # Get the NYSE calendar
    cal = mcal.get_calendar('NYSE')

    # Get the NYSE calendar's open and close times for the specified period
    schedule = cal.schedule(start_date=start_date, end_date=end_date)
    
    # Only need a list of dates when it's open (not open and close times)
    isBusinessDay = np.array(schedule.market_open.dt.strftime('%Y-%m-%d')) 
    
    # Go over all days: 
    delta = datetime.timedelta(days=1)
    start_date = datetime.datetime.strptime(start_date,"%Y-%m-%d") #datetime.date(2015, 7, 16)
    end_date = datetime.datetime.strptime(end_date,"%Y-%m-%d") #datetime.date(2023, 1, 4)
    
    # Extract days from the timedelta object
    num_days = (end_date - start_date).days + 1
    
    # Create boolean array for days being open (1) and closed (0) 
    is_open = np.zeros(num_days)
    
    # iterate over range of dates
    current_BusinessDay = isBusinessDay[0]
    count_dates = 0
    next_BusinessDay = 0
    
    while (start_date <= end_date):
    
        if start_date.strftime('%Y-%m-%d') == current_BusinessDay:
            is_open[count_dates] = True

            if current_BusinessDay == end_date_save or current_BusinessDay==isBusinessDay[-1]:
                break
            else:
                next_BusinessDay += 1
                current_BusinessDay = isBusinessDay[next_BusinessDay]
        else:
            is_open[count_dates] = False

        count_dates += 1   
        start_date += delta
        
    print(np.shape(is_open))
        
    return isBusinessDay, is_open

In [ ]:
# Define your file path and name
file_path = '/Users/manos/Documents/BDS/MLops_mod/TSLA_stock_price.csv'  # Customize the path and filename

# Save the DataFrame to CSV
data.to_csv(file_path)

print(f"Data saved to {file_path}")